In [2]:
import os
import numpy as np
import rasterio
from rasterio.windows import from_bounds



In [4]:

# Rutas de entrada
landsat_tiles_dir = './Datos/WCS_land_use/train_200218'  # Carpeta con los tiles Landsat
dem_path = './Datos/WCS_land_use/Imagery/wcs_orinoquia_srtm.tif'              # DEM grande (SRTM)
output_dir = './Datos/WCS_land_use/DEM'     # Carpeta de salida

os.makedirs(output_dir, exist_ok=True)

# Parámetros de normalización (ajusta según tu DEM)
dem_mean = 595.44
dem_std = 735.92

# Abrir DEM grande una sola vez
with rasterio.open(dem_path) as dem_src:
    for tile_name in os.listdir(landsat_tiles_dir):  # Quita tqdm
        if not tile_name.endswith('.tif'):
            continue

        tile_path = os.path.join(landsat_tiles_dir, tile_name)
        out_path = os.path.join(output_dir, tile_name)

        with rasterio.open(tile_path) as tile_src:
            landsat = tile_src.read()
            profile = tile_src.profile

            bounds = tile_src.bounds
            dem_window = from_bounds(*bounds, transform=dem_src.transform)
            dem_data = dem_src.read(1, window=dem_window, out_shape=(landsat.shape[1], landsat.shape[2]))

            dem_norm = (dem_data - dem_mean) / dem_std
            dem_norm = dem_norm.astype(np.float32)
            dem_norm = np.expand_dims(dem_norm, axis=0)

            stacked = np.concatenate([landsat, dem_norm], axis=0)

            profile.update({
                'count': stacked.shape[0],
                'dtype': 'float32',
                'compress': 'lzw'
            })

            with rasterio.open(out_path, 'w', **profile) as dst:
                dst.write(stacked)